In [4]:
import pandas as pd

In [5]:
df = pd.read_csv(r'C:\Users\BHARGAV\Desktop\clean.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 34 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   Unnamed: 0               1296675 non-null  int64  
 1   amt                      1296675 non-null  float64
 2   lat                      1296675 non-null  float64
 3   long                     1296675 non-null  float64
 4   city_pop                 1296675 non-null  int64  
 5   unix_time                1296675 non-null  int64  
 6   merch_lat                1296675 non-null  float64
 7   merch_long               1296675 non-null  float64
 8   is_fraud                 1296675 non-null  int64  
 9   transaction_year         1296675 non-null  int64  
 10  transaction_month        1296675 non-null  int64  
 11  transaction_date         1296675 non-null  int64  
 12  transaction_hour         1296675 non-null  int64  
 13  transaction_minute       1296675 non-null 

In [7]:
numeric_cols = ['amt', 'age', 'city_pop',]
df_clean = df.copy()

for col in numeric_cols:
    Q1 = df_clean[col].quantile(0.25)
    Q3 = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]

print(f"Shape after removing outliers: {df_clean.shape}")

Shape after removing outliers: (1000560, 34)


In [8]:
df_clean

,Unnamed: 0,amt,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud,transaction_year,...,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel
0,0,4.97,36.0788,-81.1781,3495,1325376018,36.011293,-82.048315,0,2019,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,107.23,48.8878,-118.2105,149,1325376044,49.159047,-118.186462,0,2019,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,45.00,46.2306,-112.1138,1939,1325376076,47.034331,-112.561071,0,2019,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,41.96,38.4207,-79.4629,99,1325376186,38.674999,-78.632459,0,2019,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,5,94.63,40.3750,-75.2045,2158,1325376248,40.653382,-76.152667,0,2019,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,1296670,15.56,37.7175,-112.4777,258,1371816728,36.841266,-111.690765,0,2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1296671,1296671,51.70,39.2667,-77.5101,100,1371816739,38.906881,-78.246528,0,2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1296672,1296672,105.93,32.9396,-105.8189,899,1371816752,33.619513,-105.130529,0,2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1296673,1296673,74.90,43.3526,-102.5411,1126,1371816816,42.788940,-103.241160,0,2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
df = df_clean

In [10]:
df = df.drop(columns=['Unnamed: 0'])

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Assume `df` is your processed DataFrame with all features and 'is_fraud' label

# 1. Prepare X and y
X = df.drop('is_fraud', axis=1)
y = df['is_fraud']

# 2. Train-test split (stratify for class imbalance)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# 3. Initialize XGBoost model
model = XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    use_label_encoder=False,
    scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum(),  # handle imbalance
    random_state=42
)

# 4. Train model
model.fit(X_train, y_train)

# 5. Predict & evaluate
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:,1]

print(classification_report(y_test, y_pred))
print(f"ROC AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")


c:\Users\BHARGAV\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:07:09] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


              precision    recall  f1-score   support

           0       1.00      0.99      1.00    199816
           1       0.20      0.83      0.32       296

    accuracy                           0.99    200112
   macro avg       0.60      0.91      0.66    200112
weighted avg       1.00      0.99      1.00    200112

ROC AUC: 0.9903


In [12]:
from sklearn.metrics import classification_report

# Predict on training set
y_train_pred = model.predict(X_train)
print("Training Classification Report:")
print(classification_report(y_train, y_train_pred))

# Predict on test set
y_test_pred = model.predict(X_test)
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))


Training Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    799264
           1       0.25      1.00      0.40      1184

    accuracy                           1.00    800448
   macro avg       0.63      1.00      0.70    800448
weighted avg       1.00      1.00      1.00    800448

Test Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00    199816
           1       0.20      0.83      0.32       296

    accuracy                           0.99    200112
   macro avg       0.60      0.91      0.66    200112
weighted avg       1.00      0.99      1.00    200112



In [13]:
# Save model to a binary file
model.save_model(r'C:\Users\BHARGAV\Desktop\Credit-Card-Transactions-Fraud-Detection\Models\xgb_model.json')  # or 'xgb_model.model'


In [14]:
for col in df.columns:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype(int)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000560 entries, 0 to 1296674
Data columns (total 33 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   amt                      1000560 non-null  float64
 1   lat                      1000560 non-null  float64
 2   long                     1000560 non-null  float64
 3   city_pop                 1000560 non-null  int64  
 4   unix_time                1000560 non-null  int64  
 5   merch_lat                1000560 non-null  float64
 6   merch_long               1000560 non-null  float64
 7   is_fraud                 1000560 non-null  int64  
 8   transaction_year         1000560 non-null  int64  
 9   transaction_month        1000560 non-null  int64  
 10  transaction_date         1000560 non-null  int64  
 11  transaction_hour         1000560 non-null  int64  
 12  transaction_minute       1000560 non-null  int64  
 13  transaction_second       1000560 non-null  int6

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1. Prepare features and target
X = df.drop(columns=['is_fraud'])
y = df['is_fraud']

# 2. Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# 3. Convert boolean columns to float (if any)
for col in X_train.columns:
    if X_train[col].dtype == 'bool':
        X_train[col] = X_train[col].astype(float)
        X_test[col] = X_test[col].astype(float)

# 4. Get numeric features list (excluding object types like strings/categories)
num_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# 5. Apply Standard Scaling
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[num_features] = scaler.fit_transform(X_train[num_features])
X_test_scaled[num_features] = scaler.transform(X_test[num_features])

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

rf = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
rf.fit(X_train_scaled, y_train)

y_pred_rf = rf.predict(X_test_scaled)
y_proba_rf = rf.predict_proba(X_test_scaled)[:, 1]

print("Random Forest Report:")
print(classification_report(y_test, y_pred_rf))
print("ROC AUC:", roc_auc_score(y_test, y_proba_rf))

KeyboardInterrupt: 

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ab = AdaBoostClassifier(n_estimators=100, random_state=42)
ab.fit(X_train_scaled, y_train)

y_pred_ab = ab.predict(X_test_scaled)
y_proba_ab = ab.predict_proba(X_test_scaled)[:, 1]

print("AdaBoost Report:")
print(classification_report(y_test, y_pred_ab))
print("ROC AUC:", roc_auc_score(y_test, y_proba_ab))


c:\Users\BHARGAV\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    199816
           1       1.00      0.00      0.01       296

    accuracy                           1.00    200112
   macro avg       1.00      0.50      0.50    200112
weighted avg       1.00      1.00      1.00    200112

ROC AUC: 0.9762295416512922


In [ ]:
import joblib

In [ ]:
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [18]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score

def train_best_xgb_scaled(X_train_scaled, y_train, X_test_scaled, y_test, random_state=42):
    # Calculate scale_pos_weight for imbalance
    scale_pos_weight = sum(y_train == 0) / sum(y_train == 1)
    
    xgb = XGBClassifier(
        objective='binary:logistic',
        eval_metric='auc',
        use_label_encoder=False,
        random_state=random_state,
        n_jobs=-1,
        tree_method='gpu_hist',  # use GPU if available; else change to 'hist'
        predictor='gpu_predictor',
        scale_pos_weight=scale_pos_weight
    )
    
    param_dist = {
        'n_estimators': [100, 200],
        'max_depth': [3, 5],
        'learning_rate': [0.05, 0.1],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0],
    }
    
    rnd_search = RandomizedSearchCV(
        estimator=xgb,
        param_distributions=param_dist,
        n_iter=5,
        scoring='roc_auc',
        cv=3,
        verbose=2,
        random_state=random_state,
        n_jobs=-1
    )
    
    rnd_search.fit(X_train_scaled, y_train)
    best_model = rnd_search.best_estimator_
    
    y_pred = best_model.predict(X_test_scaled)
    y_proba = best_model.predict_proba(X_test_scaled)[:, 1]
    
    print("Classification Report (Test Data):")
    print(classification_report(y_test, y_pred))
    print(f"ROC AUC (Test Data): {roc_auc_score(y_test, y_proba):.4f}")
    
    return best_model, rnd_search.best_params_

In [20]:
best_model, best_params = train_best_xgb_scaled(X_train_scaled, y_train, X_test_scaled, y_test)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


c:\Users\BHARGAV\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
4 fits failed out of a total of 15.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\BHARGAV\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\BHARGAV\anaconda3\Lib\site-packages\xgboost\core.py", line 729, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "c:\Users\BHARGAV\anaconda3\Lib\site-packages\xgboost\sklearn.py", line 1663, in fit
    train_dmatrix, evals = _wrap_evaluation_matrices(
                  

Classification Report (Test Data):
              precision    recall  f1-score   support

           0       1.00      0.95      0.97    199816
           1       0.03      0.97      0.05       296

    accuracy                           0.95    200112
   macro avg       0.51      0.96      0.51    200112
weighted avg       1.00      0.95      0.97    200112

ROC AUC (Test Data): 0.9879


In [22]:
# Save the model to disk
import joblib
joblib.dump(best_model, 'best_xgb_model.pkl')

['best_xgb_model.pkl']

In [23]:
best_model.save_model('best_xgb_model.json')